# Data preparation: Transactions and loans

In [74]:
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from sklearn import preprocessing
from IPython.core.interactiveshell import InteractiveShell
pd.set_option('display.max_columns',None)
%matplotlib inline

trans_df = pd.read_csv("data/trans_dev.csv",sep=";", low_memory=False)
loan_df = pd.read_csv('data/loan_dev.csv', sep=';')

## Transaction preparation



### Data quality issues

#### Noise

Nothing to report os improvements to be made.

#### Outliers

Nothing to report os improvements to do know, but we will check the outliers in the next sprints with more detail and attention.

#### Inconsistent or incorrect data 

#### Missing values

#### Duplicates

No duplicates were found in the data understanding phase. Nothing to report.

#### Inconsistent or Incorrect Data

Todo: 

Maybe we can try to search or try to reach some professional about technique/specific information about banks and accounts, but we will not do it now.



### Data pre-processing

#### Data cleaning

In [75]:
#### Data cleaning, transformation and data quality changes
trans_df.rename(columns={'date' : 'trans_date'}, inplace=True)
trans_df.rename(columns={'type' : 'trans_type'}, inplace=True)
trans_df.rename(columns={'operation' : 'trans_operation'}, inplace=True)
trans_df.rename(columns={'amount' : 'trans_amount'}, inplace=True)
trans_df.rename(columns={'balance' : 'trans_balance'}, inplace=True)
trans_df.rename(columns={'k_symbol' : 'trans_k_symbol'}, inplace=True)
trans_df.rename(columns={'bank' : 'trans_bank'}, inplace=True)
trans_df.rename(columns={'account': 'trans_account'}, inplace=True)


#as colunas bank e account não são necessárias, têm muitos valores nulos
trans_df.drop(['trans_bank', 'trans_account','trans_k_symbol' ], axis=1, inplace=True)
print(trans_df.head())



   trans_id  account_id  trans_date trans_type               trans_operation  \
0   1548749        5270      930113     credit                credit in cash   
1   1548750        5270      930114     credit  collection from another bank   
2   3393738       11265      930114     credit                credit in cash   
3   3122924       10364      930117     credit                credit in cash   
4   1121963        3834      930119     credit                credit in cash   

   trans_amount  trans_balance  
0         800.0          800.0  
1       44749.0        45549.0  
2        1000.0         1000.0  
3        1100.0         1100.0  
4         700.0          700.0  


Todo: 
- Handling Missing Values
- Handling Duplicates
- Handling Inconsistent or Incorrect Data
    - statistical-based methods to detect outliers
    - Domain knowledge
    - • Inconsistency detection

We change the names of many columns to make them more understandable and easier to work with. 

We decide to drop the columns account bank and k_symbol( now renamed to trans_account, trans_bank and trans_k_symbol) , due to the high number of missing values and more importantly, because nothing is being discovered that points they are relevant to the analysis. 

Todo: check if the missing values are relevant to the analysis.

However, for the trans_k_symbol attribute we will now drop the column, but as mentioned in the data_understanding phase later on we will treat it more carefully and make sure if we can, for example, replace with the values in some way that would not introduce bias.


##### Outliers

Todo: do it in the final
Todo: detect here outliers and on feature engineering deal with them

#### Data transformation

##### One-Hot Enconding

In [76]:
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()
trans_df['trans_operation']= label_encoder.fit_transform(trans_df['trans_operation'])

trans_df['trans_type'] = label_encoder.fit_transform(trans_df['trans_type'] )
trans_df.head()

,trans_id,account_id,trans_date,trans_type,trans_operation,trans_amount,trans_balance
0,1548749,5270,930113,0,2,800.0,800.0
1,1548750,5270,930114,0,0,44749.0,45549.0
2,3393738,11265,930114,0,2,1000.0,1000.0
3,3122924,10364,930117,0,2,1100.0,1100.0
4,1121963,3834,930119,0,2,700.0,700.0


##### Normalization

Some common strategies:
- Normalization: z-score
- Binarization / One-Hot Enconding
- Discretization
- See more

For machine learning layers:
- BATCH NORMALIZATION
- LOCAL RESPONSE NORMALIZATION
- See more




In [77]:

#Min-Max Scaling (Range-based Normalization) 
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
#Transform date into datetime , get month, day, year columns we can do feature engineering later
trans_df['trans_date'] = pd.to_datetime(trans_df['trans_date'], format='%y%m%d')
# trans_date_year = []
# trans_date_month = []
# trans_date_day = []

# for i in trans_df.index:
#     trans_date_year.append(trans_df['trans_date'][i].year)
#     trans_date_month.append(trans_df["trans_date"][i].month)
#     trans_date_day.append(trans_df['trans_date'][i].day )

# trans_df['trans_year'] = trans_date_year
# trans_df['trans_month'] = trans_date_month
# trans_df['trans_day'] = trans_date_day
#del trans_df['trans_date']

#creating normalize objects
save_trans_date = trans_df["trans_date"].copy()
save_trans_date
del trans_df["trans_date"]
normMinMaxScaler = MinMaxScaler()
normRobustScaler = RobustScaler()

#Applying the normalization techniques
trans_df_rs =  pd.DataFrame(normRobustScaler.fit_transform( trans_df),columns= trans_df.columns,)
trans_df_mms = pd.DataFrame(normMinMaxScaler.fit_transform(trans_df), columns=trans_df.columns)


In [78]:

print("MinMaxScaler normalization: \n")
trans_df_mms.head()
trans_df_mms.describe()

MinMaxScaler normalization: 



,trans_id,account_id,trans_type,trans_operation,trans_amount,trans_balance
count,396685.000000,396685.000000,396685.000000,396685.000000,396685.000000,396685.000000
mean,0.336508,0.220318,0.305458,0.682007,0.065712,0.238043
std,0.329435,0.177570,0.256677,0.263240,0.106370,0.094903
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.106391,0.095862,0.000000,0.600000,0.001476,0.173558
50%,0.214030,0.194974,0.500000,0.800000,0.022593,0.214692
75%,0.345838,0.294877,0.500000,0.800000,0.075231,0.280723
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [79]:

print("RobustScaler normalization: \n")
print(trans_df_rs.head())
print(trans_df_rs.describe())
print("\n\n")


RobustScaler normalization: 

   trans_id  account_id  trans_type  trans_operation  trans_amount  \
0  0.862365    1.346578        -1.0             -2.0     -0.180777   
1  0.862366    1.346578        -1.0             -4.0      6.715889   
2  2.954504    3.993377        -1.0             -2.0     -0.149392   
3  2.647413    3.595585        -1.0             -2.0     -0.133699   
4  0.378407    0.712583        -1.0             -2.0     -0.196469   

   trans_balance  
0      -1.356298  
1       0.656096  
2      -1.347304  
3      -1.342807  
4      -1.360795  
            trans_id     account_id     trans_type  trans_operation  \
count  396685.000000  396685.000000  396685.000000    396685.000000   
mean        0.511506       0.127344      -0.389085        -0.589964   
std         1.375817       0.892242       0.513354         1.316198   
min        -0.893850      -0.979691      -1.000000        -4.000000   
25%        -0.449529      -0.498013      -1.000000        -1.000000   
50%      

**Data transformation: final report** 


What we have done:
- One-hot encoding on categorical nominal attribute trans_operation and trans_type ( maybe also on trans_k_symbol if we decido to keep it )
- Change values withdrawal in cash to withdrawal on trans_type attribute
- change the values on trans_ammount attribute to be negative when trans_type attribute values are withdrawal
- change trans_date attribute to datetime type :year-month-day
- add trans_year attribute to the dataset getting information from trans_date
- add trans_month attribute to the dataset getting information from trans_date
- drop trans_date collumn 
- Min-Max Scaling (Range-based Normalization) not so sensitive to outliers
- Also change float values ?!?!?

Todo:
- In the future, apply : LOCAL RESPONSE NORMALIZATION and or BATCH NORMALIZATION ( ML layer )!!!
- On every dataset and the merged/final dataset check distributions and change/adapte to other types of normalization if needed
- check for normal distribution data and apply z-score normalization if needed


#### Feature engineering

Some common strategies:
- Log Transform
- Imputation
- Dealing With Dates
- Outliers: treat the outliers discovered in the data_preparation phase (data cleaning) and data understanding phase
    - DBSCAN and Isolation Forest, percentiles, z-score using data visualization on data understanding phase
- Binning
- one hot encoding ( done on data transformation phase )
- scaling : Standardization or Normalization ( done on data transformation phase using Min-MaxScaler)
- Automated Feature Engineering: Featuretools

TODO and see:
- relative values instead of absolute values
- Time Delay Embedding
- create ratios and proportions
- • express known case dependencies; use feature engineering to express the dependencies ( relationships between datasets)
- see more common techniques on feature engineering 


##### Deep Feature Synthesis

In [80]:
#Data frame with robust scaler normalization
# Automated feature engineering using featuretools
#Feature Synthesis
import featuretools as ft
from featuretools.primitives import Sum
from woodwork.logical_types import Categorical
from woodwork.logical_types import Datetime
import woodwork as ww
import data_understanding_utils as du
# Create an entityset and add the entity
# even trans_operation and trans_type are numerical, we categorize them as categorical to use DF_rsS
# append again the trans_date column
trans_df["trans_date"] = save_trans_date
trans_df_mms["trans_date"] = save_trans_date
trans_df_rs["trans_date"] = save_trans_date

trans_df.ww.init(name="ww_t_df")
trans_df_mms.ww.init(name="ww_t_df_mms")
trans_df_rs.ww.init(name="ww_t_df_rs")
trans_df.ww.set_types(logical_types= {"trans_operation": Categorical, 
                        "trans_type": Categorical, "trans_date": Datetime})
trans_df.ww.types

trans_df_mms.ww.set_types(logical_types= {"trans_operation": Categorical, 
                        "trans_type": Categorical, "trans_date": Datetime})
trans_df_mms.ww.types

trans_df_rs.ww.set_types(logical_types= {"trans_operation": Categorical, 
                        "trans_type": Categorical, "trans_date": Datetime})
trans_df_rs.ww.types

trans_df.ww.set_index("trans_id")
trans_df.ww.set_time_index("trans_date")
trans_df_mms.ww.set_index("trans_id")
trans_df_mms.ww.set_time_index("trans_date")
trans_df_rs.ww.set_index("trans_id")
trans_df_rs.ww.set_time_index("trans_date")
#mean year, month balance for each account
save_balance_month_year_df = trans_df.groupby(by=["account_id",trans_df["trans_date"].dt.year,trans_df["trans_date"].dt.month])['trans_balance'].mean()
save_balance_month_year_df_mms = trans_df_mms.groupby(by=["account_id",trans_df_mms["trans_date"].dt.year,trans_df_mms["trans_date"].dt.month])['trans_balance'].mean()
save_balance_month_year_df_rs = trans_df_rs.groupby(by=["account_id",trans_df_rs["trans_date"].dt.year,trans_df_rs["trans_date"].dt.month])['trans_balance'].mean()

In [81]:
#print(trans_df.head())
# trans_df.drop('trans_id', axis=1, inplace=True)
# print(trans_df.head())
# print(trans_df.isnull().sum())
# du.check_duplicates(trans_df,"transaction",trans_df.columns)
entitySet_t_df = ft.EntitySet(id ="trans_df")
entitySet_t_df
entitySet_t_df = entitySet_t_df.add_dataframe( dataframe= trans_df ,dataframe_name= "trans_df" , index= "trans_id",
                        time_index = 'trans_date')

# # Here we will only use on transaction dataframe
# #Create cutoff time to use DFS

# #Check also max_depth , verbose and n_jobs
ft_trans_df_matrix , features_trans__df_defs = ft.dfs( entityset = entitySet_t_df,  target_dataframe_name= trans_df.ww.name,
                                                     
                                      max_depth = 1, verbose = 1, n_jobs = 1)
    
ft_trans_df_matrix.sort_values(by=["account_id","YEAR(trans_date)","MONTH(trans_date)","DAY(trans_date)"] ,inplace=True, ascending=True)

ft_trans_df_matrix.head(5)

# #same for trans_df_mms and trans_df_rs




/home/wolfcuanhamarws/.local/lib/python3.10/site-packages/featuretools/entityset/entityset.py:754: UserWarning: A Woodwork-initialized DataFrame was provided, so the following parameters were ignored: index, time_index, dataframe_name
  warnings.warn(


Built 9 features
Elapsed: 00:05 | Progress: 100%|██████████


,account_id,trans_type,trans_operation,trans_amount,trans_balance,DAY(trans_date),MONTH(trans_date),WEEKDAY(trans_date),YEAR(trans_date)
trans_id,,,,,,,,,
1,1,0,2,1000.0,1000.0,24,3,4,1995
5,1,0,0,3679.0,4679.0,13,4,3,1995
199,1,0,2,12600.0,17279.0,23,4,6,1995
3530438,1,0,5,19.2,17298.2,30,4,6,1995
6,1,0,0,3679.0,20977.2,13,5,5,1995


In [82]:
# trans_df_mms

entitySet_t_df_mms = ft.EntitySet(id ="trans_df_mms")
entitySet_t_df_mms
entitySet_t_df_mms = entitySet_t_df_mms.add_dataframe( dataframe= trans_df_mms ,dataframe_name= "trans_df_mms" , index= "trans_id",
                        time_index = 'trans_date')

# # Here we will only use on transaction dataframe
# #Create cutoff time to use DFS

# #Check also max_depth , verbose and n_jobs
ft_trans_df_mms_matrix , features_trans_df_mms_defs = ft.dfs( entityset = entitySet_t_df_mms, target_dataframe_name= trans_df_mms.ww.name,
                                                     
                                           max_depth = 1, verbose = 1, n_jobs = 1)
    
ft_trans_df_mms_matrix.sort_values(by=["account_id","YEAR(trans_date)","MONTH(trans_date)","DAY(trans_date)"] ,inplace=True, ascending=True)

ft_trans_df_mms_matrix.head(5)




Built 9 features


/home/wolfcuanhamarws/.local/lib/python3.10/site-packages/featuretools/entityset/entityset.py:754: UserWarning: A Woodwork-initialized DataFrame was provided, so the following parameters were ignored: index, time_index, dataframe_name
  warnings.warn(


Elapsed: 00:05 | Progress: 100%|██████████


,account_id,trans_type,trans_operation,trans_amount,trans_balance,DAY(trans_date),MONTH(trans_date),WEEKDAY(trans_date),YEAR(trans_date)
trans_id,,,,,,,,,
0.000000,0.0,0.0,0.4,0.011574,0.070307,24,3,4,1995
0.000001,0.0,0.0,0.0,0.042581,0.088038,13,4,3,1995
0.000054,0.0,0.0,0.4,0.145833,0.148761,23,4,6,1995
0.958594,0.0,0.0,1.0,0.000222,0.148854,30,4,6,1995
0.000001,0.0,0.0,0.0,0.042581,0.166584,13,5,5,1995


In [83]:

# trans_df_rs





entitySet_t_df_rs = ft.EntitySet(id ="trans_df_rs")
entitySet_t_df_rs
entitySet_t_df_rs = entitySet_t_df_rs.add_dataframe( dataframe= trans_df_rs ,dataframe_name= "trans_df_rs" , index= "trans_id",
                        time_index = 'trans_date')

# # Here we will only use on transaction dataframe
# #Create cutoff time to use DFS

# #Check also max_depth , verbose and n_jobs
ft_trans_df_rs_matrix , features_trans_df_rs_defs = ft.dfs( entityset = entitySet_t_df_rs, target_dataframe_name= trans_df_rs.ww.name , max_depth = 1, verbose = 1, n_jobs = 1)
    
ft_trans_df_rs_matrix.sort_values(by=["account_id","YEAR(trans_date)","MONTH(trans_date)","DAY(trans_date)"] ,inplace=True, ascending=True)

ft_trans_df_rs_matrix.head(5)


/home/wolfcuanhamarws/.local/lib/python3.10/site-packages/featuretools/entityset/entityset.py:754: UserWarning: A Woodwork-initialized DataFrame was provided, so the following parameters were ignored: index, time_index, dataframe_name
  warnings.warn(


Built 9 features
Elapsed: 00:05 | Progress: 100%|██████████


,account_id,trans_type,trans_operation,trans_amount,trans_balance,DAY(trans_date),MONTH(trans_date),WEEKDAY(trans_date),YEAR(trans_date)
trans_id,,,,,,,,,
-0.893850,-0.979691,-1.0,-2.0,-0.149392,-1.347304,24,3,4,1995
-0.893846,-0.979691,-1.0,-4.0,0.271008,-1.181857,13,4,3,1995
-0.893626,-0.979691,-1.0,-2.0,1.670930,-0.615226,23,4,6,1995
3.109517,-0.979691,-1.0,1.0,-0.303303,-0.614363,30,4,6,1995
-0.893845,-0.979691,-1.0,-4.0,0.271008,-0.448916,13,5,5,1995


In [84]:
entitySet_t_df


Entityset: trans_df
  DataFrames:
    ww_t_df [Rows: 396685, Columns: 7]
  Relationships:
    No relationships

In [85]:
entitySet_t_df_mms


Entityset: trans_df_mms
  DataFrames:
    ww_t_df_mms [Rows: 396685, Columns: 7]
  Relationships:
    No relationships

In [86]:
entitySet_t_df_rs

Entityset: trans_df_rs
  DataFrames:
    ww_t_df_rs [Rows: 396685, Columns: 7]
  Relationships:
    No relationships

In [87]:
#join all the transaction dataframes with the year_month_balance dataframe
# ft_trans_df_matrix.join(save_balance_month_year_df, on="account_id", how= "inner")
# ft_trans_df_mms_matrix.join(save_balance_month_year_df, on="account_id", how= "inner")
# ft_trans_df_mms_matrix.join(save_balance_month_year_df, on="account_id", how= "inner")

##### Tuning Deep Feature Synthesis

**Final report:**

Done:
- Dealing with dates : Transform date into datetime , get month, day, year columns we can do feature engineering later. This was done on data transformation phase so we can normalize in a way  that in the end we can run all cells at once .
- Scaling was done: MinMaxScaler() and  RobustScaler().
- Use Deep Feature Synthesis to create new features from the existing ones. We will do it here and on the final dataset, after merging all relevant atributes of all data datasets.

TODO: 
- Categorical imputation : can be usefull for the trans_k_symbol column !! if we decide to keep it
- Scaling: StandardScaler() and Normalization: z-score if we find to have normal distribution in the data
- merge the mean, year, month balance with the final ft_matrix and mergo with loans dataset to get more feature engineering
- merge


#### Data and Dimensionality Reduction

**Some Strategies**
• Feature Selection
• Principal Components Analysis (PCA)
• Singular Value Decomposition (SVD)

TODO:
- See others

#### Sampling Data

In [ ]:
''' from ._adasyn import ADASYN
from ._random_over_sampler import RandomOverSampler
from ._smote import SMOTE
from ._smote import BorderlineSMOTE
from ._smote import KMeansSMOTE
from ._smote import SVMSMOTE
from ._smote import SMOTENC
from ._smote import SMOTEN
'''
from sklearn.model_selection import train_test_split, StratifiedKFold
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import KMeansSMOTE
def split_data(X, y, test_size):
    return train_test_split(X, y, test_size=0.3, random_state=42, shuffle=False)

def smote_sampling(X_train, y_train):
    sm = SMOTE(random_state=42)
    return sm.fit_resample(X_train, y_train)

#### Feature Extraction

### Export new transaction data to csv file

In [88]:
ft_trans_df_matrix.to_csv('refined/transaction_df.csv')
ft_trans_df_mms_matrix.to_csv('refined/transaction_df_mms.csv')
ft_trans_df_mms_matrix.to_csv('refined/transaction_df_rs.csv')



# Loan preparation

Todo:
    transactions:as transactions vão passar apenas a ter: média do balanço das transações por mês de cada ano,balanço mensal ,soma do nº de transações mensal e anual?, taxa de esforço mensal. Isso assim reduz imenso a dimensão da merge data e não perde muita informação .
    - loans: for every account: number of clients, loan_ammount, loan_duration,loan_payments(year), loan_paymentos_month = loan_payments/12, loan_duration -> cruzar com a transactions, client e district e account_frequency ( check better this)
    - client; mix with the other datasets and: get age of client when loan_age =x and loan_status 1 or 0, gender
    - district: check better this
    - client is the most importante object to compare with every other feature 
        - client:gender,account number of loans, number of people on the same account, number of accounts ?, the other informations above, average salayr for his district, and other info in district, number of transactions, ....
        Focus on: for every pair client,account -> info relationship with the other tables
     - What to do about the missing values ?? , outliers and duplicates ?? and inconsistent or incorrect data ?? check

     - WHat to do for clients with more than one account ?? and for more than one loan ?? (Example: one loand payed, other not)-> count number of loan payed ?check this !!!
     - condensate information: on row for every client,account,loan -> number of loans, number of people on the same account, number of accounts ?, the other informations above, average salayr for his district, and other info in district, number of transactions, .... just stay with the owners and count the number of clients on the same account(it seems logic: the owner is who have always something to say if any loan is requested)